In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
import os

os.chdir("..")

In [4]:
from fairseq.models.transformer import TransformerModel

In [5]:
source_data_path = "experiments/doc-indices/sp-cl-Europarl.en-et.docs.test.both"
target_data_path = "experiments/doc-indices/sp-cl-Europarl.en-et.docs.test.both"
cluster_ids_path = "experiments/en_et_concat60/nmt-clusters-sent/Europarl.test.clust.nmt.sent"

experts_id2path = {
    "0": "experiments/en_et_Europarl_ft",
    "1": "experiments/en_et_Europarl_ft",
    "2": "experiments/en_et_Europarl_ft",
    "3": "experiments/en_et_Europarl_ft"
}

In [9]:
id2model = {}
for id, path in experts_id2path.items():
    model = TransformerModel.from_pretrained(path, 'checkpoint100.pt', 'experiments/bin-data-en-et-Europarl-ft/')
    model.cuda()
    model.eval()
    id2model[id] = model

In [10]:
sources = []
targets = []
doc_ids = []
with open(source_data_path) as both: 
    for line in both:
        doc_id, src, tgt = line.strip().split('\t')
        doc_ids.append(doc_id)
        sources.append(src)
        targets.append(tgt) 
        
cluster_ids = []
with open(cluster_ids_path) as f: 
    for line in f:
        cluster_ids.append(line.strip())


In [11]:
%%time

translations = []
i = 0
for id, src in zip(cluster_ids, sources):
    
    model = id2model[id] 

#     for k, v in src.items():
#         src[k] = v.to(model.device)

    with torch.no_grad():
        hyp = model.translate(src, beam=5)

    translations.append(hyp)

    if i % 100 == 0:
        print(f"{i}/{len(sources)} sentences translated")

    i += 1

0/3157 sentences translated
100/3157 sentences translated
200/3157 sentences translated
300/3157 sentences translated
400/3157 sentences translated
500/3157 sentences translated
600/3157 sentences translated
700/3157 sentences translated
800/3157 sentences translated
900/3157 sentences translated
1000/3157 sentences translated
1100/3157 sentences translated
1200/3157 sentences translated
1300/3157 sentences translated
1400/3157 sentences translated
1500/3157 sentences translated
1600/3157 sentences translated
1700/3157 sentences translated
1800/3157 sentences translated
1900/3157 sentences translated
2000/3157 sentences translated
2100/3157 sentences translated
2200/3157 sentences translated
2300/3157 sentences translated
2400/3157 sentences translated
2500/3157 sentences translated
2600/3157 sentences translated
2700/3157 sentences translated
2800/3157 sentences translated
2900/3157 sentences translated
3000/3157 sentences translated
3100/3157 sentences translated
CPU times: user 8min

In [12]:
from sacrebleu import corpus_bleu as _corpus_bleu

In [13]:
def corpus_bleu(sys_stream, ref_streams):
    bleu = _corpus_bleu(sys_stream, ref_streams, tokenize="none")
    return bleu.score

In [14]:
def detok(s):
    return ''.join(s.split()).replace('▁', ' ').strip()

In [21]:
from sentencepiece import SentencePieceProcessor

In [24]:
SentencePieceProcessor("data-prep/preproc-models/syscl-en-et.model")

<sentencepiece.SentencePieceProcessor; proxy of <Swig Object of type 'sentencepiece::SentencePieceProcessor *' at 0x7fc031a691e0> >

In [23]:
targets_detok[2]

'Andsime selleks aruteluks selle tähtsuse tõttu palju aega ja loodame, et meil õnnestub jätkata üle meie aja, et lõpetada infotund.'

In [15]:
targets_detok = [detok(t) for t in targets]
hyps_detok = [detok(t) for t in translations]

In [16]:
corpus_bleu(targets_detok, [hyps_detok])

24.03040001067341

In [17]:
corpus_bleu(targets, [translations])

32.12108793535469

In [18]:
corpus_bleu(targets_detok, [hyps_detok])

24.03040001067341

In [19]:
corpus_bleu(targets, [translations])

32.12108793535469

In [52]:
basename = source_data_path.split('/')[-1]
with open(f"hypothesis/hyp-{basename}.bpe.et", 'w') as f:
    for s in translated:
        f.write(f"{s}\n")

In [18]:
a = []
with open("experiments/fairseq-data-en-et-Europarl-ft/test.et") as f:
    for l in f:
        a.append(l.strip())
        

In [19]:
b = []
with open("gr-hyp.europarl.et") as f:
    for l in f:
        b.append(l.strip())
        

In [20]:
corpus_bleu(a, [b])

31.286223180045237

In [22]:
da = [detok(l) for l in a]
db = [detok(l) for l in b]

In [23]:
corpus_bleu(da, [db])

23.395436862030135